# Definitive PxS predict commands

In this notebook, I generate the terminal commands for the final experiment with PxS.

In [1]:
from aaai20.io import (
    build_filesystem,
    default_prefix_exp_fn_suffix,
    filename_config,
    filename_cli_commands,
    filename_script,
    filename_logs,
    filename_query
)

from aaai20.cmd_gen import (
    all_fnames_in_dir,
    generate_config_and_log_fname,
    generate_df_commands,
    extract_idx_from_fnames,
    default_start_idx,
    prepare_fs,
)


import json
import os
import pandas as pd
import datetime
import numpy as np

# Functions

In [2]:
def generate_predict_config(
    dataset="nltcs",
    model_keyword="default",
    exp_keyword="test-exp",
    script="run_mercs",
    exp_idx=0,
    **predict_config
):

    default_predict_config = dict(inferce_algorithm="EPIS_sampling")

    predict_config = {**default_predict_config, **predict_config}

    config = dict(
        dataset=dataset,
        model_keyword=model_keyword,
        exp_keyword=exp_keyword,
        exp_idx=exp_idx,
        predict_config=predict_config,
        script=script,
    )

    return config

def generate_fit_config(
    dataset="nltcs",
    model_keyword="default",
    exp_keyword="test-exp",
    script="run_mercs",
    exp_idx=0,
    **fit_config
):

    default_fit_config = dict()

    fit_config = {**default_fit_config, **fit_config}

    config = dict(
        dataset=dataset,
        model_keyword=model_keyword,
        exp_keyword=exp_keyword,
        exp_idx=exp_idx,
        fit_config=fit_config,
        script=script,
    )

    return config

# RUN PXS

I generate the bunch of commands that will be executed to run all the queries. The  difficulty is that each dataset has a different amount of queries.

In [3]:
def count_queries(dataset, query_keyword="default", fs=None):
    # Load queries
    fn_qry = filename_query("nltcs", suffix="default", fs=fs)
    q_codes = np.load(fn_qry)
    return q_codes.shape[0]

## Functions

## Demo

Local demonstration of my system. I will also use GNU Parallel.

In [4]:
fs = build_filesystem()
starai_ds = os.listdir(fs['datasets-starai'])
starai_ds

['tretail',
 'pumsb_star',
 'bnetflix',
 'kdd',
 'bbc',
 'nltcs',
 'plants',
 'kosarek',
 'baudio',
 'tmovie',
 'voting',
 'book',
 'dna',
 'cwebkb',
 'jester',
 'msnbc',
 'msweb',
 'cr52',
 'c20ng',
 'accidents',
 'ad']

In [5]:
# Fixed parameters
exp_keyword = "predict-pxs"
script = "run_pxs"
model_keyword = "default"
fs = build_filesystem()
prepare_fs(exp_keyword, script, fs)
timeout = datetime.timedelta(hours=0.5).seconds
start_idx = default_start_idx(fs, script=script)
nb_queries = 100  # Every dataset has 100 queries

# Predict parameters
inference_algorithm = "EPIS_sampling"

# Grid definition
phase_space = starai_ds

# Inits
df_commands = pd.DataFrame()
fn_cfg = []
fn_log = []
q_idx = []
fn_script = filename_script(script, kind="predict")

for p_idx, p in enumerate(phase_space):
    for single_q_idx in range(nb_queries):
        # Init
        exp_idx = start_idx + p_idx

        # Generate config
        predict_config = dict(inference_algorithm=inference_algorithm)

        config = generate_predict_config(
            dataset=p,
            script=script,
            model_keyword=model_keyword,
            exp_keyword=exp_keyword,
            exp_idx=exp_idx,
            **predict_config
        )

        # Save config
        single_fn_cfg, single_fn_log = generate_config_and_log_fname(
            config,
            kind="predict",
            exp_fn_fields=["inference_algorithm"],
            qry_idx=single_q_idx,
        )
        with open(single_fn_cfg, "w") as f:
            json.dump(config, f, indent=4)

        fn_cfg.append(single_fn_cfg)
        fn_log.append(single_fn_log)
        q_idx.append(single_q_idx)

df_commands = generate_df_commands(fn_script, fn_cfg, fn_log, timeout, q_idx=q_idx)

fn_commands = filename_cli_commands(exp_keyword)
df_commands.to_csv(fn_commands, index=False)

In [6]:
df_commands.head()

,script_fname,config_fname,log_fname,timeout,q_idx
0,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/cli/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,1800,26
1,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/cli/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,1800,2
2,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/cli/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,1800,15
3,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/cli/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,1800,88
4,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/cli/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,/cw/dtaijupiter/NoCsBack/dtai/elia/aaai20/out/...,1800,36
